In [5]:
#Non blocking - get_message()

import redis
r = redis.Redis("localhost", 38443, decode_responses = True)

In [50]:
subscriber = r.pubsub()
subscriber.subscribe('my_q')

while True:
    message = subscriber.get_message()
    if message:
        if message['type'] == 'message':
            print("Received message: ", message['data'])

            #Exit the loop as soon as 'STOP' message is received.
            if message['data'] == 'STOP':
                print("Exiting the loop....")
                break
subscriber.close()

Received message:  STOP
Exiting the loop....


In [6]:
subscriber = r.pubsub()
subscriber.subscribe('my_q')


for message in subscriber.listen():
    if message['type'] == 'message':
        print("Received message: ", message['data'])

        #Exit the loop as soon as 'STOP' message is received.
        if message['data'] == 'STOP':
            print("Exiting the loop....")
            break
subscriber.close()

Received message:  STOP
Exiting the loop....


In [ ]:
# Class

redis_config = {
    "host": "localhost",
    "port": 38443,
    "decode_responses": True,
    "channel": "my_q"
}

class redisclass:
    def __init__(self, host, port, decode_responses, channel):
        self.host = host
        self.port = port
        self.decode_responses = decode_responses
        self.channel = channel
        self.subscriber = None
    
    def connect(self):
        self.r = redis.Redis(self.host, self.port, decode_responses = self.decode_responses)
        return self.r
    
    def subscribe_(self):
        self.connect()
        self.subscriber = self.r.pubsub()
        self.subscriber.subscribe(self.channel)
        return self.subscriber

    def close(self):
        if self.subscriber:
            self.subscriber.close()


In [ ]:
# Usage with get_message()

re = redisclass(**redis_config)
re.connect()
re_subscribe = re.subscribe_()
while True:
    message = re_subscribe.get_message()
    if message:
        if message['type'] == 'message':
            print(message['data'])
            if message['data'] == 'STOP':
                print("Exiting the loop...")
                break


STOP
Exiting the loop...


In [29]:
re.close()

In [30]:
### As it is compulsory to close the subsriber after operation is done. So I have made custom context manager.
### In this case, we will make connection and subscribe to channel and then return the object in with context.

redis_config = {
    "host": "localhost",
    "port": 38443,
    "decode_responses": True,
    "channel": "my_q"
}

class redisclass:
    def __init__(self, host, port, decode_responses, channel):
        self.host = host
        self.port = port
        self.decode_responses = decode_responses
        self.channel = channel
        self.subscriber = None
    
    def connect(self):
        self.r = redis.Redis(self.host, self.port, decode_responses = self.decode_responses)
        return self.r
    
    def subscribe_(self):
        r = self.connect()
        self.subscriber = r.pubsub()
        self.subscriber.subscribe(self.channel)
        return self.subscriber
    
    def __enter__(self):
        return self.subscribe_()
    
    def __exit__(self, exc_type, exc_value, traceback):
        if self.subscriber:
            self.subscriber.close()
            print("PubSub connection closed.")


In [31]:
with redisclass(**redis_config) as r:
    while True:
        message = r.get_message()
        if message:
            if message['type'] == 'message':
                print(message['data'])
                if message['data'] == 'STOP':
                    print("Exiting the loop...")
                    break

Hare Krishna
STOP
Exiting the loop...
PubSub connection closed.


In [ ]:
### As it is compulsory to close the subsriber after operation is done. So I have made custom context manager.
### In this case, we will make connection return the connection object in 'with context'. and on that object we will call the subscribe method.

import redis

redis_config = {
    "host": "localhost",
    "port": 38443,
    "decode_responses": True,
    "channel": "my_q"
}

class RedisClass:
    def __init__(self, host, port, decode_responses, channel):
        self.host = host
        self.port = port
        self.decode_responses = decode_responses
        self.channel = channel
        self.r = None
        self.subscriber = None
    
    def connect(self):
        if not self.r:
            self.r = redis.Redis(self.host, self.port, decode_responses=self.decode_responses)
        return self.r
    
    def subscribe_(self):
        if not self.r:
            self.connect()
        self.subscriber = self.r.pubsub()
        self.subscriber.subscribe(self.channel)
        return self.subscriber
    
    def __enter__(self):
        self.connect()
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        if self.subscriber:
            self.subscriber.close()
            print("PubSub connection closed.")

In [39]:
with RedisClass(**redis_config) as r:
    subs = r.subscribe_()
    while True:
        message = subs.get_message()
        if message:
            if message['type'] == 'message':
                print(message['data'])
                if message['data'] == 'STOP':
                    print("Exiting the loop...")
                    break

HARE kRISHNA
STOP
Exiting the loop...
PubSub connection closed.


In [4]:
with RedisClass(**redis_config) as r:
    subs = r.subscribe_()
    for message in subs.listen():
        if message['type'] == 'message':
            print(f"Received message: {message['data']}")
            if message['data'] == 'STOP':
                print("Exiting the loop...")
                break

Received message: STOP
Exiting the loop...
PubSub connection closed.
